In [1]:
import pandas as pd
import numpy as np
import os
import  matplotlib.pyplot as plt
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

In [2]:
country  = 'tanzania'
err_type = 'scaled_mse'
scale    = 1
# Directory after running predict-v3.ipynb.
path_to_results = '../tanzania_v6_7_crop_sales/'
path_to_tables  = '../Tables_7_v2/'

In [11]:
country  = 'uganda'
err_type = 'scaled_mse'
scale    = 1
# Directory after running predict-v3.ipynb.
path_to_results = '../uganda_v2_crop_sales/'
path_to_tables  = '../Tables_uganda_v2/'

In [12]:
outcomes = sorted(set([x.split('___')[0] for x in os.listdir(path_to_results) if 'coef' not in x and 'clus' not in x and 'plot' not in x and 'lock' not in x and 'elbow' not in x]))

model_names = {
'All variables':'All Inputs',
'Highly correlated policy and non-policy variables':'Relevant Inputs',
'Policy variables':'Policy Inputs',
}

index_row = []

In [4]:
outcomes

['crop_sales']

In [5]:

def summary_mse(year,outcome,var_spec):
    
    mse = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==False)][err_type].values[0]

    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==True)].index#[err_type].idxmin()

    mse_segments = list(raw_mse.loc[idx][err_type])

    segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))

    all_series = pd.DataFrame()
    for segment, mse_segment in zip(segments, mse_segments):
        series = pd.Series([
            year,
            outcome.replace('_',' ').title(),
            'LASSO + '+model_names[var_spec],
            np.round(mse/scale,3),
            segment.replace('_',' ').title(),
            np.round(mse_segment/scale,3),
            np.round(mse/mse_segment-1,3)],
            index=[
            'Year',
            'Outcome',
            'Model',
            'Mean Sq. Error',
            'Household Segmentation',
            'Mean Sq. Error Segmented',
            '% Reduction in Error',
            ])
        all_series = pd.concat([all_series, series], 1)
    
    return all_series
        
            
def summary_coef(year,outcome,var_spec,is_clustered=True):
    global index_row
    # Get Segment Corresponding to Min MSE
    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==is_clustered)].index#[err_type].idxmin()

    best_segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))
    all_series = pd.DataFrame()
    for best_segment in best_segments:
#         &(raw_coef['segmentation']==var_spec[:-2]
        best_var_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).idxmax()

#     &(raw_coef['segmentation']==var_spec[:-2])
        best_coef_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float)[best_var_all]
        segments = int(math.pow(2, len(best_segment.split(','))))
        best_var = {}
        best_coef = {}
        best_of = 5
        for k in range(4):
#             try:
            best_c = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).nlargest(best_of)
#             best_t = best_c.apply(lambda x: x.index.tolist(), axis=1)
#             print (best_t.as_matrix())
#             best_coefs =  best_c.as_matrix()
            best_vars = best_c.index.tolist()
            best_coefs = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].iloc[0][best_vars].as_matrix()
            print (best_coefs)

            row = []
            index_row =[]
            for ix, t, i in zip(range(best_of), best_vars, best_coefs):
                row += [
                    t.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                    np.round(i,3)
                ]
                index_row += ['Var ' + str(ix), 'Coef ' +  str(ix)]
#             except:
#                 print (var_spec, best_segment, k)
            series = pd.Series([
                year,
                outcome.replace('_',' ').title(),
                'LASSO + '+model_names[var_spec],
                best_var_all.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                np.round(best_coef_all,3),
                best_segment.replace('___ethiopia_'+str(year), '').title()] + row,index=[
                'Year',
                'Outcome',
                'Model',
                'Most Relevant Input',
                'Coef. Most Relevant Input',
                'Household Segmentation']
                    + index_row)
            all_series = pd.concat([all_series, series], 1)
        
    return all_series

def summary_coef_unsegmented(year,outcome,var_spec):
    global index_row
    # Get Segment Corresponding to Min MSE
    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==False)].index#[err_type].idxmin()

    all_series = pd.DataFrame()

    best_var_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
        regex='policy').iloc[0].astype(float).apply(np.abs).idxmax()

#     &(raw_coef['segmentation']==var_spec[:-2])
    best_coef_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
        regex='policy').iloc[0].astype(float)[best_var_all]
    best_var = {}
    best_coef = {}
    best_of = 5

    best_c = raw_coef.loc[
    (raw_coef['inputs']==var_spec)&\
    (raw_coef['model']=='Lasso_' + var_spec)].filter(
    regex='policy').iloc[0].astype(float).apply(np.abs).nlargest(best_of)
#             best_t = best_c.apply(lambda x: x.index.tolist(), axis=1)
#             print (best_t.as_matrix())
#             best_coefs =  best_c.as_matrix()
    best_vars = best_c.index.tolist()
    best_coefs = raw_coef.loc[
    (raw_coef['inputs']==var_spec)&\
    (raw_coef['model']=='Lasso_' + var_spec)].iloc[0][best_vars].as_matrix()
    print (best_coefs)

    row = []
    index_row =[]
    for ix, t, i in zip(range(best_of), best_vars, best_coefs):
        row += [
            t.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
            np.round(i,3)
        ]
        index_row += ['Var ' + str(ix), 'Coef ' +  str(ix)]
#             except:
#                 print (var_spec, best_segment, k)
    series = pd.Series([
        year,
        outcome.replace('_',' ').title(),
        'LASSO + '+model_names[var_spec],
        best_var_all.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
        np.round(best_coef_all,3),
        ''] + row,index=[
        'Year',
        'Outcome',
        'Model',
        'Most Relevant Input',
        'Coef. Most Relevant Input',
        'Household Segmentation']
            + index_row)
    all_series = pd.concat([all_series, series], 1)
        
    return all_series
        

In [13]:
# Run this after all the ones above to generate best coefficient for all cluster's regressions.
import math
all_mse  = pd.DataFrame()
all_coef = pd.DataFrame()
all_raw_mse = pd.DataFrame()

all_raw_coef = pd.DataFrame()
years = [2014]

for outcome in outcomes:
    
    for year in years:
        
        try:
            raw_mse = pd.read_csv(
            path_to_results+outcome+'___'+'output'+'.csv', #'___'+country+'_'+str(year)+
            usecols=['clustered','input',err_type,'segment'])
        except:
            continue
        
        raw_mse['output'] = outcome
        all_raw_mse = pd.concat([all_raw_mse, raw_mse], 0)
        all_raw_mse['segment'] = all_raw_mse['segment'].fillna('').apply(lambda x: x.replace('___ethiopia_'+str(year), ''))
        raw_coef = pd.read_csv(
        path_to_results+'coef_'+outcome+'___'+'output'+'.csv',index_col=0) # +'___'+country+'_'+str(year)
        
        # Variables On Which Segmentation is Performed
        raw_coef['segmentation'] = raw_coef['model'].apply(
        lambda x:x.replace('Lasso_','').replace('OrdinaryLeastSquares_','').replace('clustered_','').replace('segmented_','').replace('___' + country + '_'+str(year), '')[:-2])

        # Segment Index
        raw_coef['output'] = outcome
        raw_coef['year'] = year
        raw_coef['segment'] = raw_coef['model'].apply(
        lambda x:x.split('_')[-1])
        all_raw_coef = pd.concat([all_raw_coef, raw_coef], 0)

        for var_spec in sorted(model_names):
#             try:
            all_coef = pd.concat([all_coef, summary_coef(year,outcome,var_spec)],1)
#             all_coef = pd.concat([all_coef, summary_coef_unsegmented(year,outcome,var_spec)],1)
#             except:
#                 continue
            
all_coef = all_coef.T.set_index(['Outcome','Year','Model'])[[
'Household Segmentation',
'Most Relevant Input', 
'Coef. Most Relevant Input',] + index_row].copy()

all_coef.to_excel(path_to_tables+'coef_'+country+'.xlsx', engine='openpyxl')

[0.29928887390957859 0.26377049424142252 0.25294295345382634
 0.15566758442172446 0.066819313599292274]
[0.15307216442698091 0.1094167703401059 0.10085330074579919
 -0.065121697975628223 0.028139107965118652]
[0.13788145885749226 0.13058058960727345 0.12516013863724115
 -0.065649246541606504 0.051682413888501685]
[0.15968209681468698 0.10299685333105643 0.10099558624602577
 -0.035686210454668334 0.0094452236201564913]
[0.2833099533279414 0.26172724383930651 0.25033828432022259
 0.15459980408707266 -0.067055575421646946]
[0.15937333576952059 0.12874315733394434 0.071488188440935976
 0.070517434590786174 -0.066851530038506857]
[0.1353784159254609 0.13235233259033025 0.12628385833526787
 -0.067669003795566049 0.054624755875367352]
[0.13653496808749108 -0.11080580334621276 0.109207857726346
 0.095480629960225119 -0.053345358351455319]
